# Test for processing Canvas data: Dev

This notebook demonstrates possible data processing and exploration of the Canvas data, using the OEA_py class notebook. 

Most of the data processing done in this notebook are also achieved by executing the Canvas module main pipeline. This notebook is designed as an alternate approach to the same processing, as well as module data exploration and visualization. 

The steps are clearly outlined below:
1. Set the workspace,
2. Land Canvas Module Higher Ed. Test Data,
3. Pre-Process Canvas Module Test Data,
4. Ingest the Canvas Module Test Data,
5. Refine the Canvas Module Test Data, 
6. Demonstrate Lake Database Queries/Final Remarks, and
7. Appendix

In [1]:
%run OEA_py

StatementMeta(, 2, -1, Finished, Available)

2023-05-19 16:14:30,335 - OEA - INFO - Now using workspace: dev
2023-05-19 16:14:30,336 - OEA - INFO - OEA initialized.


In [2]:
# 1) set the workspace (this determines where in the data lake you'll be writing to and reading from).
# You can work in 'dev', 'prod', or a sandbox with any name you choose.
# For example, Sam the developer can create a 'sam' workspace and expect to find his datasets in the data lake under oea/sandboxes/sam
oea.set_workspace('dev')

StatementMeta(spark3p3sm, 2, 3, Finished, Available)

2023-05-19 16:14:32,424 - OEA - INFO - Now using workspace: dev


## 2.) Land Canvas Module Higher Ed. Test Data

Directory: ```GitHub.com (raw data) -> stage1/Transactional/canvas```

The code block below lands 10 OEA Canvas module test data tables, formatted as Canvas Higher Ed. data in your data lake. 

Canvas test data tables landed in stage 1:
 1. **accounts**
 2. **assignments**
 3. **assignment_submissions**
 4. **assignment_submission_summary**
 5. **courses**
 6. **course_sections**
 7. **enrollments**
 8. **enrollment_terms**
 9. **roles**
 10. **users** 

**To-Do's:**
 - Correct the test dataset as needed
 - Remove top codeblock for when launching on OEA.

In [50]:
# In this example we pull Canvas HEd test json data files from github and land it in oea/dev/stage1/Transactional/canvas/v2.0
import datetime
currentDate = datetime.datetime.now()
currentDateTime = currentDate.strftime("%Y-%m-%d %H-%M-%S")
data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/accounts.json').text
oea.land(data, 'canvas/v2.0/accounts', 'accounts_hed_test_data.json', oea.SNAPSHOT_BATCH_DATA, currentDateTime)
data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/courses.json').text
oea.land(data, 'canvas/v2.0/courses', 'courses_hed_test_data.json', oea.SNAPSHOT_BATCH_DATA, currentDateTime)
data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/roles.json').text
oea.land(data, 'canvas/v2.0/roles', 'roles_hed_test_data.json', oea.SNAPSHOT_BATCH_DATA, currentDateTime)
data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/course_sections.json').text
oea.land(data, 'canvas/v2.0/course_sections', 'coursesections_hed_test_data.json', oea.SNAPSHOT_BATCH_DATA, currentDateTime)
data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/users.json').text
oea.land(data, 'canvas/v2.0/users', 'users_hed_test_data.json', oea.SNAPSHOT_BATCH_DATA, currentDateTime)
# normally, these three tables should be landed as delta_batch_data but since functionality is limited for processing delta data, we assume they're snapshot for now.
#data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/assignment_submission_summary.json').text
#oea.land(data, 'canvas/v2.0/assignment_sumbission_summary', 'assignsubsummary_hed_test_data.csv', oea.SNAPSHOT_BATCH_DATA, currentDateTime)
data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/enrollments.json').text
oea.land(data, 'canvas/v2.0/enrollments', 'enrollments_hed_test_data.json', oea.SNAPSHOT_BATCH_DATA, currentDateTime)
data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/enrollment_terms.json').text
oea.land(data, 'canvas/v2.0/enrollment_terms', 'enrollmentterms_hed_test_data.json', oea.SNAPSHOT_BATCH_DATA, currentDateTime)

#data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/assignment_submissions.json').text
#oea.land(data, 'canvas/v2.0/assignment_submissions', 'assignsubmissions_hed_test_data.csv', oea.ADDITIVE_BATCH_DATA, currentDateTime)
#data = requests.get('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/hed_test_data/assignments.json').text
#oea.land(data, 'canvas/v2.0/assignments', 'assignments_hed_test_data.csv', oea.ADDITIVE_BATCH_DATA, currentDateTime)

StatementMeta(spark3p3sm, 2, 51, Finished, Available)

'stage1/Transactional/canvas/v2.0/enrollment_terms/snapshot_batch_data/rundate=2023-05-19 18-19-11/enrollmentterms_hed_test_data.json'

## 3.) Pre-Process Canvas Module Test Data

Directory: ```stage1/Transactional/canvas -> stage1/Transactional/canvas```

This step is responsible for pre-processing the Canvas module test data from stage1 back to stage1.

The code blocks in this step read in the original JSON tables using the ```pd.read_json(..., lines=True)``` function, performs any ad hoc data conversions, and writes the table to stage1 as a CSV.

**To-Do's:**
 - Check if this is to be adopted as a standard for JSONs oriented as records (one JSON row represents a row in the df).

In [51]:
# x) this step pre-processing the canvas data through reading in the JSONs as records, corrects any schema discepancies and then writes out the df as a CSV in stage1.
import datetime
currentDate = datetime.datetime.now()
currentDateTime = currentDate.strftime("%Y-%m-%d %H-%M-%S")

def clean_data_lake_latest(source_path):
    """only house the latest rundate folder compared to the old data (which were JSONs)"""
    latest_folder = oea.get_latest_folder(source_path)
    items = mssparkutils.fs.ls(oea.to_url(source_path))
    for item in items:
        if item.name != latest_folder:
            logger.info('file removal path: ' + item.path + ' with item: ' + item.name)
            oea.rm_if_exists(source_path + '/' + item.name)
            logger.info('Successfully removed folder: ' + item.name + ' from path: ' + item.path)
        else:
            logger.info('Kept folder: ' + item.name + ' from path: ' + item.path)
    logger.info('Finished cleaning data lake to house only the latest folder')

def write_canvas_json_as_csv(write_filepath,batch_type,df):
    df.coalesce(1).write.save(oea.to_url(f'{write_filepath}/{batch_type}_batch_data/rundate={currentDateTime}'), format='csv', mode='overwrite', header='true', mergeSchema='true')

def preprocess_canvas_dataset(tables_source):
    items = oea.get_folders(tables_source)
    for item in items: 
        table_path = tables_source +'/'+ item
        #batch_type = oea.get_folders(table_path)
        pdf = pd.read_json(oea.to_url(f'{table_path}/snapshot_batch_data/*/*.json'),lines=True)
        df = spark.createDataFrame(pdf)
        if item == 'accounts':
            df = df.withColumn('parent_account_id', df['parent_account_id'].cast(LongType()))
            write_canvas_json_as_csv(table_path,'snapshot',df)
            #clean_data_lake_latest(f'{table_path}/snapshot_batch_data')
        else:
            write_canvas_json_as_csv(table_path,'snapshot',df)
        clean_data_lake_latest(f'{table_path}/snapshot_batch_data')
        new_table_path = f'{table_path}/snapshot_batch_data/rundate={currentDateTime}'
        oea.rm_if_exists(new_table_path + '/_SUCCESS', False)
        #if batch_type == 'snapshot_batch_data':
        #    pdf = pd.read_json(oea.to_url(f'{table_path}/snapshot_batch_data/*/*.json'),lines=True)
        #    df = spark.createDataFrame(pdf)
        #    if item == 'accounts':
        #        df = df.withColumn('parent_account_id', df['parent_account_id'].cast(LongType()))
        #        write_canvas_json_as_csv(table_path,'snapshot',df)
                #clean_data_lake_latest(f'{table_path}/snapshot_batch_data')
        #    else:
        #        write_canvas_json_as_csv(table_path,'snapshot',df)
                #clean_data_lake_latest(f'{table_path}/snapshot_batch_data')
            # now write to CSV and clean stage1 of the lake
            #write_canvas_json_as_csv(table_path,'snapshot',df)
            #clean_data_lake_latest(f'{table_path}/snapshot_batch_data')
        #elif batch_type == 'delta_batch_data':
        #    pdf = pd.read_json(oea.to_url(f'{table_path}/delta_batch_data/*/*.json'),lines=True)
        #    df = spark.createDataFrame(pdf)
        #    write_canvas_json_as_csv(table_path,'delta',df)
        #    clean_data_lake_latest(f'{table_path}/delta_batch_data')
        #elif batch_type == 'additive_batch_data':
        #    pdf = pd.read_json(oea.to_url(f'{table_path}/additive_batch_data/*/*.json'),lines=True)
        #    df = spark.createDataFrame(pdf)
        #    write_canvas_json_as_csv(table_path,'additive',df)
        #    clean_data_lake_latest(f'{table_path}/additive_batch_data')
        logger.info('Pre-processed table: ' + item + ' from: ' + table_path)
    logger.info('Finished pre-processing Canvas tables')

StatementMeta(spark3p3sm, 2, 52, Finished, Available)

In [52]:
# set the version number and pre-process the dataset
version = '2.0'
preprocess_canvas_dataset(f'stage1/Transactional/canvas/v{version}')

StatementMeta(spark3p3sm, 2, 53, Finished, Available)

2023-05-19 18:19:28,439 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://stoeacisd3v08oct.blob.core.windows.net/oea?restype=REDACTED&comp=REDACTED&prefix=REDACTED&delimiter=REDACTED&include=REDACTED'
Request method: 'GET'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': 'b1484280-f671-11ed-bb1c-000d3a17ca77'
    'User-Agent': 'azsdk-python-storage-blob/12.14.1 Python/3.10.6 (Linux-4.15.0-1164-azure-x86_64-with-glibc2.27)'
    'Authorization': 'REDACTED'
No body was attached to the request
2023-05-19 18:19:28,480 - azure.core.pipeline.policies.http_logging_policy - INFO - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/xml'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '00b335fa-a01e-0028-687e-8ac95f000000'
    'x-ms-client-request-id': 'b1484280-f671-11ed-bb1c-000d3a17ca

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a futu

## 4.) Ingest the Canvas Module Test Data

Directory: ```stage1/Transactional/canvas -> stage2/Ingested/canvas```

This step ingests the Canvas module test data from stage1 to stage2/Ingested.

The code blocks in this step ingest the data using the ```oea.ingest()``` function as normal.

**To-Do's:**
 - Check if Canvas test data accurately reflects actual (production) Canvas data.

In [53]:
# 3) The next step is to ingest the batch data into stage2
# Note that when you run this the first time, you'll see an info message like "Number of new inbound rows processed: 2".
# If you run this a second time, the number of inbound rows processed will be 0 because the ingestion uses spark structured streaming to keep track of what data has already been processed.
oea.ingest(f'canvas/v2.0/accounts', 'id')
#oea.ingest(f'canvas/v2.0/assignments', 'id')
#oea.ingest(f'canvas/v2.0/assignment_submissions', 'anonymous_id')
#oea.ingest(f'canvas/v2.0/assignment_submission_summary', 'assignment_id')
oea.ingest(f'canvas/v2.0/courses', 'id')
oea.ingest(f'canvas/v2.0/course_sections', 'id')
oea.ingest(f'canvas/v2.0/enrollments', 'id')
oea.ingest(f'canvas/v2.0/enrollment_terms', 'id')
oea.ingest(f'canvas/v2.0/roles', 'id')
oea.ingest(f'canvas/v2.0/users', 'id')

StatementMeta(spark3p3sm, 2, 54, Finished, Available)

2023-05-19 18:20:06,650 - OEA - INFO - Ingesting from: stage1/Transactional/canvas/v2.0/accounts, batch type of: snapshot, source data format of: csv
source_path is: abfss://oea@stoeacisd3v08oct.dfs.core.windows.net/dev/stage1/Transactional/canvas/v2.0/accounts/snapshot_batch_data/rundate=2023-05-19 18-19-17
2023-05-19 18:20:07,777 - py4j.java_gateway - INFO - Callback Server Starting
2023-05-19 18:20:07,778 - py4j.java_gateway - INFO - Socket listening on ('127.0.0.1', 36129)
2023-05-19 18:20:09,449 - py4j.java_gateway - INFO - Callback Connection ready to receive messages
2023-05-19 18:20:09,455 - py4j.java_gateway - INFO - Received command c on object id p0
2023-05-19 18:20:26,033 - OEA - INFO - Number of new inbound rows processed: 180
2023-05-19 18:20:34,965 - OEA - INFO - Ingesting from: stage1/Transactional/canvas/v2.0/courses, batch type of: snapshot, source data format of: csv
source_path is: abfss://oea@stoeacisd3v08oct.dfs.core.windows.net/dev/stage1/Transactional/canvas/v2.

640

In [54]:
# 3.5) Now you can run queries against the auto-generated "lake database" with the ingested Canvas data.
df = spark.sql("select * from ldb_dev_s2i_canvas_v2p0.course_sections")
display(df.limit(10))

StatementMeta(spark3p3sm, 2, 55, Finished, Available)

SynapseWidget(Synapse.DataFrame, bedf0aa4-03fe-435e-928c-62bb77593d01)

## 5.) Refine the Canvas Module Test Data

Directory: ```stage2/Ingested/canvas -> stage2/Refined/canvas```

This step then refines the Canvas test data from stage2/Ingested to stage2/Refined, using the metadata.csv. This step is responsible for pseudonymization, which preserves sensitive student information by either hashing or masking the sensitive columns. 

Tables are separated into either ```stage2/Refined/canvas/v2.0/general``` or ```stage2/Refined/canvas/v2.0/sensitive```, depending on whether each table is pseudonymized or has a sensitive column-hashing/masking mapping, respectively.


In [55]:
def refine_canvas(entity_path, metadata=None, primary_key='id'):
    source_path = f'stage2/Ingested/{entity_path}'
    primary_key = oea.fix_column_name(primary_key) # fix the column name, in case it has a space in it or some other invalid character
    path_dict = oea.parse_path(source_path)
    sink_general_path = path_dict['entity_parent_path'].replace('Ingested', 'Refined') + '/general/' + path_dict['entity']
    sink_sensitive_path = path_dict['entity_parent_path'].replace('Ingested', 'Refined') + '/sensitive/' + path_dict['entity'] + '_lookup'
    if not metadata:
        all_metadata = oea.get_metadata_from_path(path_dict['entity_parent_path'])
        metadata = all_metadata[path_dict['entity']]

    df_changes = oea.get_latest_changes(source_path, sink_general_path)
    spark_schema = oea.to_spark_schema(metadata)
    df_changes = oea.modify_schema(df_changes, spark_schema)        

    if df_changes.count() > 0:
        df_pseudo, df_lookup = oea.pseudonymize(df_changes, metadata)
        oea.upsert(df_pseudo, sink_general_path, primary_key) # todo: remove this assumption that the primary key will always be hashed during pseduonymization
        oea.upsert(df_lookup, sink_sensitive_path, primary_key)    
        oea.add_to_lake_db(sink_general_path)
        oea.add_to_lake_db(sink_sensitive_path)
        logger.info(f'Processed {df_changes.count()} updated rows from {source_path} into stage2/Refined')
    else:
        logger.info(f'No updated rows in {source_path} to process.')
    return df_changes.count()

StatementMeta(spark3p3sm, 2, 56, Finished, Available)

In [56]:
# 4) this step refines the data through the use of metadata (this is where the pseudonymization of the data occurs).
def refine_canvas_dataset(tables_source):
    items = oea.get_folders(tables_source)
    for item in items: 
        table_path = tables_source +'/'+ item
        if item == 'metadata.csv':
            logger.info('ignore metadata processing, since this is not a table to be ingested')
        else:
            try:
                if item == 'accounts':
                    refine_canvas('canvas/v2.0/accounts', metadata[item], 'id_pseudonym')
                elif item == 'users':
                    refine_canvas('canvas/v2.0/users', metadata[item], 'id_pseudonym')
                else:
                    refine_canvas('canvas/v2.0/' + item, metadata[item], 'id')
            except AnalysisException as e:
                # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
                pass
            
            logger.info('Refined table: ' + item + ' from: ' + table_path)
    logger.info('Finished refining Canvas tables')

StatementMeta(spark3p3sm, 2, 57, Finished, Available)

In [57]:
metadata = oea.get_metadata_from_url('https://raw.githubusercontent.com/cstohlmann/oea-canvas-module/main/test_data/metadata.csv')
refine_canvas_dataset('stage2/Ingested/canvas/v2.0')

StatementMeta(spark3p3sm, 2, 58, Finished, Available)

2023-05-19 18:26:47,288 - OEA - INFO - Processed 180 updated rows from stage2/Ingested/canvas/v2.0/accounts into stage2/Refined
2023-05-19 18:26:47,591 - OEA - INFO - Refined table: accounts from: stage2/Ingested/canvas/v2.0/accounts
2023-05-19 18:26:51,525 - OEA - INFO - Refined table: course_sections from: stage2/Ingested/canvas/v2.0/course_sections
2023-05-19 18:26:55,337 - OEA - INFO - Refined table: courses from: stage2/Ingested/canvas/v2.0/courses
2023-05-19 18:26:58,905 - OEA - INFO - Refined table: enrollment_terms from: stage2/Ingested/canvas/v2.0/enrollment_terms
2023-05-19 18:27:02,996 - OEA - INFO - Refined table: enrollments from: stage2/Ingested/canvas/v2.0/enrollments
2023-05-19 18:27:06,611 - OEA - INFO - Refined table: roles from: stage2/Ingested/canvas/v2.0/roles
2023-05-19 18:27:17,008 - OEA - INFO - Processed 640 updated rows from stage2/Ingested/canvas/v2.0/users into stage2/Refined
2023-05-19 18:27:17,239 - OEA - INFO - Refined table: users from: stage2/Ingested/c

## 6.) Demonstrate Lake Database Queries/Final Remarks

In [60]:
# non-hashed primary keys are not automatically added to the lake db - add these tables
oea.add_to_lake_db('stage2/Refined/canvas/v2.0/general/courses')
oea.add_to_lake_db('stage2/Refined/canvas/v2.0/general/course_sections')
oea.add_to_lake_db('stage2/Refined/canvas/v2.0/general/enrollments')
oea.add_to_lake_db('stage2/Refined/canvas/v2.0/general/enrollment_terms')
oea.add_to_lake_db('stage2/Refined/canvas/v2.0/general/roles')

StatementMeta(spark3p3sm, 2, 61, Finished, Available)

In [61]:
# 5) Now you can query the refined data tables in the lake db
df = spark.sql("select * from ldb_dev_s2r_canvas_v2p0.enrollments")
display(df)
df.printSchema()
df = spark.sql("select * from ldb_dev_s2r_canvas_v2p0.users")
display(df)
df.printSchema()
# You can use the "lookup" table for joins (people with restricted access won't be able to perform this query because they won't have access to data in the "sensitive" folder in the data lake)
df = spark.sql("select e.course_section_id, e.type, e.workflow_state, u.id_pseudonym, u.name \
                from ldb_dev_s2r_canvas_v2p0.enrollments e, ldb_dev_s2r_canvas_v2p0.users u where e.user_id_pseudonym = u.id_pseudonym")
display(df.limit(10))

StatementMeta(spark3p3sm, 2, 62, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8b291fdd-def3-4ce1-9e1f-8c6657b118fe)

root
 |-- id: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- workflow_state: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- start_at: timestamp (nullable = true)
 |-- end_at: timestamp (nullable = true)
 |-- completed_at: timestamp (nullable = true)
 |-- self_enrolled: boolean (nullable = true)
 |-- last_activity_at: timestamp (nullable = true)
 |-- course_section_id: integer (nullable = true)
 |-- user_id_pseudonym: string (nullable = true)



SynapseWidget(Synapse.DataFrame, e05f7a30-f648-4fbf-9439-ba6197897f4c)

root
 |-- id_pseudonym: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time_zone: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- locale: string (nullable = true)
 |-- workflow_state: string (nullable = true)
 |-- sortable_name: string (nullable = true)
 |-- school_name: string (nullable = true)
 |-- school_position: string (nullable = true)
 |-- public: boolean (nullable = true)
 |-- global_canvas_id_pseudonym: string (nullable = true)



SynapseWidget(Synapse.DataFrame, 57a439f4-b56d-4d83-9aae-c51b406b5aa0)

In [19]:
# Run this cell to reset this example (deleting all the example Canvas data in your workspace)
oea.rm_if_exists('stage1/Transactional/canvas')
oea.rm_if_exists('stage2/Ingested/canvas')
oea.rm_if_exists('stage2/Refined/canvas')
oea.drop_lake_db('ldb_dev_s2i_canvas_v2p0')
oea.drop_lake_db('ldb_dev_s2r_canvas_v2p0')

## Appendix

In [ ]:
# generate an initial metadata file for manual modification
metadata = oea.create_metadata_from_lake_db('ldb_dev_s2i_canvas_v2p0')
dlw = DataLakeWriter(oea.to_url('stage1/Transactional/canvas'))
dlw.write('metadata.csv', metadata)

In [ ]:
# Create a sql db for the ingested Canvas data
oea.create_sql_db('stage2/Ingested/canvas')

In [13]:
oea.create_sql_db('stage2/Refined/canvas')

StatementMeta(spark3p2sm, 20, 14, Finished, Available)

-- Create a new sql script then execute the following in it:
IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = 'sdb_dev_s2r_moodle')
BEGIN
  CREATE DATABASE sdb_dev_s2r_moodle;
END;
GO
USE sdb_dev_s2r_moodle;
GO

CREATE OR ALTER VIEW v4.1 AS
  SELECT * FROM OPENROWSET(BULK 'abfss://oea@stoeacisd3v08kw1.dfs.core.windows.net/dev/stage2/Refined/moodle/v4.1', FORMAT='delta') AS [r];
GO

